# **ABABA**

*This paper* 

| [1. Introduction](#1.-Introduction) | [2. Literature Review](#2.-Literature-Review) | [3. Research Question](#3.-Research-Question) | [4. Data](#4.-Data) | [5. Methodology](#5.-Methodology) | [6. Results and Discussion](#6.-Results-and-Discussion) | [7. Conclusion](#7.-Conclusion) | [Bibliography](#Bibliography) |

---

## 1. Introduction


In [1]:
import time
start_time = time.time()

### 1.1 adaf

## 2. Literature Review

## 3. Research Question

## 4. Data

## 5. Methodology

## 6. Results and Discussion

## 7. Conclusion

In [2]:
end_time = time.time()
run_time = end_time - start_time
print("Total run time: ", run_time, "seconds")

Total run time:  0.017493724822998047 seconds


*(word count: )*

## Bibliography